In [1]:
import numpy as np
from ckks.ckks_parameters import CKKSParameters
from ckks.ckks_key_generator import CKKSKeyGenerator
from ckks.ckks_encryptor import CKKSEncryptor
from ckks.ckks_decryptor import CKKSDecryptor
from ckks.ckks_evaluator import CKKSEvaluator
from ckks.ckks_encoder import CKKSEncoder
from fft.fft import FFT1D, IFFT1D

print("Setting up CKKS parameters and generating keys...")
poly_degree = 8192
ciph_modulus = 1 << 800  
big_modulus = 1 << 1200
scaling_factor = 1 << 30
params = CKKSParameters(
    poly_degree=poly_degree,
    ciph_modulus=ciph_modulus,
    big_modulus=big_modulus,
    scaling_factor=scaling_factor
)
keygen = CKKSKeyGenerator(params)
public_key = keygen.public_key
secret_key = keygen.secret_key
relin_key = keygen.relin_key
encoder = CKKSEncoder(params)
encryptor = CKKSEncryptor(params, public_key, secret_key)
decryptor = CKKSDecryptor(params, secret_key)
evaluator = CKKSEvaluator(params)

data_len = poly_degree // 2
message_length = 2000
print("Keys generated with poly_degree=", poly_degree, "and cipher modulus=2^800.")

Setting up CKKS parameters and generating keys...
Keys generated with poly_degree= 8192 and cipher modulus=2^800.


In [ ]:
def convolution(x, h):
    """
    x and h are the same size
    """
    n = len(x)
    y = np.zeros(n, dtype=complex)
    for k in range(n):
        for m in range(n):
            y[k] += x[m] * h[(k - m) % n]
    return y

In [3]:
print("Generating and embedding two messages of length", message_length, "...")
orig_msg1 = np.random.random(message_length) + 1j * np.random.random(message_length)
orig_msg2 = np.random.random(message_length) + 1j * np.random.random(message_length)
vA = np.zeros(data_len, dtype=complex)
vB = np.zeros(data_len, dtype=complex)
vA[:message_length] = orig_msg1
vB[:message_length] = orig_msg2
print("Embedded vectors vA and vB of length", data_len)

Generating and embedding two messages of length 2000 ...
Embedded vectors vA and vB of length 4096


In [4]:
print("\nProcessing Message 1...")
domA = FFT1D(vA)
print("  FFT done for Message 1.")
plainA = encoder.encode(domA, params.scaling_factor)
print("  Encoding done for Message 1.")
ctA = encryptor.encrypt(plainA)
print("  Encryption done for Message 1.")
ptA = decryptor.decrypt(ctA)
print("  Decryption done for Message 1.")
decA = encoder.decode(ptA)
print("  Decoding done for Message 1.")
recA = IFFT1D(decA)
print("  IFFT done for Message 1.")
successA = np.allclose(vA, recA, atol=1e-1, rtol=1e-1)
print("Message 1 recovery successful:", successA)

print("\nProcessing Message 2...")
domB = FFT1D(vB)
print("  FFT done for Message 2.")
plainB = encoder.encode(domB, params.scaling_factor)
print("  Encoding done for Message 2.")
ctB = encryptor.encrypt(plainB)
print("  Encryption done for Message 2.")
ptB = decryptor.decrypt(ctB)
print("  Decryption done for Message 2.")
decB = encoder.decode(ptB)
print("  Decoding done for Message 2.")
recB = IFFT1D(decB)
print("  IFFT done for Message 2.")
successB = np.allclose(vB, recB, atol=1e-1, rtol=1e-1)
print("Message 2 recovery successful:", successB)


Processing Message 1...
  FFT done for Message 1.
  Encoding done for Message 1.
  Encryption done for Message 1.
  Decryption done for Message 1.
  Decoding done for Message 1.
  IFFT done for Message 1.
Message 1 recovery successful: True

Processing Message 2...
  FFT done for Message 2.
  Encoding done for Message 2.
  Encryption done for Message 2.
  Decryption done for Message 2.
  Decoding done for Message 2.
  IFFT done for Message 2.
Message 2 recovery successful: True


In [5]:
print("\nTesting homomorphic addition...")
ct_sum = evaluator.add(ctA, ctB)
print("  Homomorphic addition done.")
pt_sum = decryptor.decrypt(ct_sum)
print("  Decryption of sum done.")
dec_sum = encoder.decode(pt_sum)
print("  Decoding of sum done.")
time_sum = IFFT1D(dec_sum)
print("  IFFT of sum done.")
true_sum = vA + vB
add_success = np.allclose(time_sum, true_sum, atol=1e-1, rtol=1e-1)
print("Addition recovery successful:", add_success)


Testing homomorphic addition...
  Homomorphic addition done.
  Decryption of sum done.
  Decoding of sum done.
  IFFT of sum done.
Addition recovery successful: True


In [6]:
print("\nTesting homomorphic convolution...")
ct_prod = evaluator.multiply(ctA, ctB, relin_key)
print("  Homomorphic multiplication done.")
pt_prod = decryptor.decrypt(ct_prod)
print("  Decryption of product done.")
dec_prod = encoder.decode(pt_prod)
print("  Decoding of product done.")
time_conv = IFFT1D(dec_prod)
print("  IFFT of product done.")
true_conv = convolution(vA, vB)
mult_success = np.allclose(time_conv, true_conv, atol=1e-1, rtol=1e-1)
print("Multiplication (convolution) recovery successful:", mult_success)


Testing homomorphic convolution...
  Homomorphic multiplication done.
  Decryption of product done.
  Decoding of product done.
  IFFT of product done.
Multiplication (convolution) recovery successful: True
